### Тестовое задание "Классификация цветков ириса"
<br><b>Выполнил:</b> <i>Альчаков В.В. гр. AD11872/3<i>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


from matplotlib import pyplot
%matplotlib inline

### Загрузка тестового набора

In [2]:
iris = pd.read_csv("iris.csv")
iris = iris.drop(['Id'], axis = 1)
iris = shuffle(iris)
iris

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
54,6.5,2.8,4.6,1.5,Iris-versicolor
1,4.9,3.0,1.4,0.2,Iris-setosa
138,6.0,3.0,4.8,1.8,Iris-virginica
144,6.7,3.3,5.7,2.5,Iris-virginica
31,5.4,3.4,1.5,0.4,Iris-setosa
...,...,...,...,...,...
9,4.9,3.1,1.5,0.1,Iris-setosa
23,5.1,3.3,1.7,0.5,Iris-setosa
133,6.3,2.8,5.1,1.5,Iris-virginica
67,5.8,2.7,4.1,1.0,Iris-versicolor


### Кодирование признаков

In [3]:
lb = LabelEncoder()
lb.fit_transform(iris['Species'])

array([1, 0, 2, 2, 0, 1, 2, 0, 2, 1, 2, 1, 1, 2, 0, 1, 2, 0, 1, 0, 2, 1,
       0, 0, 2, 0, 2, 1, 1, 0, 0, 0, 1, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 0,
       1, 2, 1, 1, 1, 1, 1, 2, 2, 0, 0, 1, 0, 1, 0, 1, 2, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 2, 2, 2, 2, 1, 2, 0, 2, 1, 0, 0, 2, 2, 0, 2, 0,
       1, 0, 1, 1, 2, 0, 2, 2, 1, 0, 2, 2, 2, 1, 1, 0, 0, 1, 1, 0, 0, 2,
       2, 0, 2, 2, 2, 2, 1, 2, 0, 1, 1, 2, 0, 0, 2, 1, 0, 1, 2, 0, 2, 1,
       1, 1, 1, 2, 1, 1, 2, 2, 0, 2, 1, 2, 2, 0, 0, 2, 1, 0])

In [4]:
iris['Species'] = lb.fit_transform(iris['Species'])
iris

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
54,6.5,2.8,4.6,1.5,1
1,4.9,3.0,1.4,0.2,0
138,6.0,3.0,4.8,1.8,2
144,6.7,3.3,5.7,2.5,2
31,5.4,3.4,1.5,0.4,0
...,...,...,...,...,...
9,4.9,3.1,1.5,0.1,0
23,5.1,3.3,1.7,0.5,0
133,6.3,2.8,5.1,1.5,2
67,5.8,2.7,4.1,1.0,1


### Разбивка данных на тестовую и проверочную выборки

In [5]:
X = iris[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
y = pd.get_dummies(iris['Species']).values # One hot encoding
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 100, shuffle = True)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(120, 4)
(120, 3)
(30, 4)
(30, 3)


### Нормализация данных

In [6]:
normalizer = tf.keras.layers.Normalization(axis = -1)
normalizer.adapt(np.array(X))

### Определяем структуру нейронной сети

In [7]:
#model = tf.keras.Sequential([
#    tf.keras.layers.Dense(10, activation='relu'),
#    tf.keras.layers.Dense(10, activation='relu'),
#    tf.keras.layers.Dense(3, activation='softmax')
#])

n_features = X.shape[1] # Размерность X
model = Sequential()
model.add(normalizer)
model.add(Dense(10, activation = 'relu', input_shape = (n_features, )))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))
model.summary()


#model = Sequential()
#model.add(normalizer)
#model.add(Dense(10, activation = 'relu', input_shape = (n_features, )))
#model.add(Dense(10, activation = 'relu'))
#model.add(Dense(3, activation = 'softmax'))
#model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 4)                9         
 n)                                                              
                                                                 
 dense (Dense)               (None, 10)                50        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 3)                 33        
                                                                 
Total params: 202
Trainable params: 193
Non-trainable params: 9
_________________________________________________________________


In [8]:
#model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [9]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 50, verbose = 1, validation_data = (X_test, y_test))

Epoch 1/100
3/3 [==============================] - 0s 73ms/step - loss: 0.9341 - accuracy: 0.5750 - val_loss: 0.8392 - val_accuracy: 0.7667
Epoch 2/100
3/3 [==============================] - 0s 11ms/step - loss: 0.9171 - accuracy: 0.6083 - val_loss: 0.8202 - val_accuracy: 0.7667
Epoch 3/100
3/3 [==============================] - 0s 11ms/step - loss: 0.9002 - accuracy: 0.6333 - val_loss: 0.8021 - val_accuracy: 0.7667
Epoch 4/100
3/3 [==============================] - 0s 10ms/step - loss: 0.8850 - accuracy: 0.6500 - val_loss: 0.7842 - val_accuracy: 0.7667
Epoch 5/100
3/3 [==============================] - 0s 10ms/step - loss: 0.8692 - accuracy: 0.6667 - val_loss: 0.7673 - val_accuracy: 0.7667
Epoch 6/100
3/3 [==============================] - 0s 11ms/step - loss: 0.8544 - accuracy: 0.6667 - val_loss: 0.7511 - val_accuracy: 0.7667
Epoch 7/100
3/3 [==============================] - 0s 11ms/step - loss: 0.8398 - accuracy: 0.6667 - val_loss: 0.7354 - val_accuracy: 0.7667
Epoch 8/100
3/3 [===

3/3 [==============================] - 0s 10ms/step - loss: 0.4133 - accuracy: 0.8417 - val_loss: 0.2756 - val_accuracy: 0.9333
Epoch 60/100
3/3 [==============================] - 0s 11ms/step - loss: 0.4080 - accuracy: 0.8417 - val_loss: 0.2712 - val_accuracy: 0.9333
Epoch 61/100
3/3 [==============================] - 0s 10ms/step - loss: 0.4033 - accuracy: 0.8417 - val_loss: 0.2668 - val_accuracy: 0.9333
Epoch 62/100
3/3 [==============================] - 0s 11ms/step - loss: 0.3984 - accuracy: 0.8417 - val_loss: 0.2626 - val_accuracy: 0.9333
Epoch 63/100
3/3 [==============================] - 0s 10ms/step - loss: 0.3935 - accuracy: 0.8417 - val_loss: 0.2583 - val_accuracy: 0.9333
Epoch 64/100
3/3 [==============================] - 0s 11ms/step - loss: 0.3888 - accuracy: 0.8417 - val_loss: 0.2541 - val_accuracy: 0.9333
Epoch 65/100
3/3 [==============================] - 0s 10ms/step - loss: 0.3842 - accuracy: 0.8417 - val_loss: 0.2501 - val_accuracy: 0.9333
Epoch 66/100
3/3 [========

In [10]:
loss, accuracy = model.evaluate(X_test, y_test, verbose = 0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.13572309911251068
Test accuracy: 1.0


In [11]:
y_pred = model.predict(X_test)
y_pred

1/1 [==============================] - 0s 62ms/step


array([[1.9365843e-02, 1.0811683e-01, 8.7251729e-01],
       [1.2517206e-01, 7.3848879e-01, 1.3633914e-01],
       [9.8936242e-01, 1.0001039e-02, 6.3656218e-04],
       [9.6973605e-02, 8.7989753e-01, 2.3128916e-02],
       [9.6823096e-01, 3.1317100e-02, 4.5186013e-04],
       [4.2412286e-03, 3.9356612e-02, 9.5640212e-01],
       [1.4962642e-01, 4.4418773e-01, 4.0618587e-01],
       [9.6880686e-01, 3.1029666e-02, 1.6343695e-04],
       [1.1995279e-01, 7.0345122e-01, 1.7659596e-01],
       [2.7174405e-03, 3.6558196e-02, 9.6072435e-01],
       [9.7827071e-01, 2.1530444e-02, 1.9882745e-04],
       [4.1287155e-03, 3.9105937e-02, 9.5676541e-01],
       [1.9508766e-02, 1.1743400e-01, 8.6305720e-01],
       [1.1831442e-01, 8.3292025e-01, 4.8765261e-02],
       [9.7806996e-01, 2.1787444e-02, 1.4263170e-04],
       [9.8413020e-01, 1.5593982e-02, 2.7580588e-04],
       [9.8336339e-01, 1.6147071e-02, 4.8950606e-04],
       [3.1580436e-03, 3.7215140e-02, 9.5962685e-01],
       [9.5914936e-01, 4.061

In [12]:
actual = np.argmax(y_test, axis = 1)
predicted = np.argmax(y_pred, axis = 1)
print(f"Actual: {actual}")
print(f"Pred-d: {predicted}")

Actual: [2 1 0 1 0 2 1 0 1 2 0 2 2 1 0 0 0 2 0 2 1 0 0 0 0 0 2 2 2 1]
Pred-d: [2 1 0 1 0 2 1 0 1 2 0 2 2 1 0 0 0 2 0 2 1 0 0 0 0 0 2 2 2 1]
